<a href="https://colab.research.google.com/github/Sunnn-y/ICR_Project/blob/main/jobplanet_ABSA_%EB%B0%9C%EC%A0%84%EA%B0%80%EB%8A%A5%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정
런타임 - 런타임 유형 변경 - GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Colab 파이썬 버전 확인
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
# 필요한 라이브러리 설치
!pip install mxnet                    # mxnet: 딥러닝 프레임워크
!pip install gluonnlp pandas tqdm     # gluonnlp: 오픈소스 딥러닝 기반의 자연어 처리 툴킷
!pip install sentencepiece            # sentencepiece: pre-tokenization을 필요로 하지 않는 tokenizer
!pip install transformers             # transformer
!pip install torch                    # torch

In [ ]:
# https://github.com/SKTBrain/KoBERT 의 파일을 Colab으로 다운로드
!pip install 'git+https://git@github.com/SKTBrain/KoBERT.git@master'
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ha49l6vh
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ha49l6vh
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.15.18-py2.py3-none-any.whl (129 kB)
  Using cached mxnet-1.7.0.post2-py2.py3-none-manylinux2014_x86_64.whl (54.7 MB)
INFO: pip is looking at multiple versions of kobert to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement onnxruntime<=1.8.0,==1.8.0 (from kobert) (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3)
ERROR: No matching distribution found for onnxruntime<=1.8.0,==1.8.0
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/p

In [ ]:
# 필요한 라이브러리 import
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# 모델링

In [ ]:
df_labeled = pd.read_csv('/content/drive/MyDrive/ICR_project/잡플래닛_라벨링/jobplanet_labeled_발전가능성.csv')
df_labeled.head()

,Unnamed: 0,문장,감정
0,0,업무강도가 강하지만 그만큼 높은 급여를 기대할 수 있음,중립
1,1,대전지역에 몇 없는 중견기업 중 하나 신입연봉치곤 괜찮은 연봉,중립
2,2,연차 자유롭게 사용 가능 함 다만 현장이 바쁠때는 사용 불가 팀장이랑 조율 해야함 ...,중립
3,3,제공되는 중식으로 식비를 절약할수있음 자유로운 휴식시간과 단순한 문서보조 작업만 해...,긍정
4,4,1대전에서 연봉이 높은편이다. 2점심을 준다. 3음,중립


In [ ]:
df = df_labeled.copy()

In [ ]:
df = df[df['감정'] != '중립']

In [ ]:
# 필요 없는 칼럼(Unnamed: 0) 제거
df = df.drop(columns=['Unnamed: 0'])
df.head(1)

,문장,감정
3,제공되는 중식으로 식비를 절약할수있음 자유로운 휴식시간과 단순한 문서보조 작업만 해...,긍정


In [ ]:
df.head()

,문장,감정
3,제공되는 중식으로 식비를 절약할수있음 자유로운 휴식시간과 단순한 문서보조 작업만 해...,긍정
20,노사다 사관학교 멀티플레어 역할을 할 수있음. 다양한 업무를 경험할 수 있음,긍정
23,1 많은 토목회사들이 현장근무는 격주 토요일 근무를 하는데 반해 이곳은 주5일 근무...,긍정
38,좋은 사람들 많습니다 무엇보다 연봉이 높으며 배울 것이 많습니다,긍정
40,공공발주사업 많이 따와서 업무를 빡세게 배울 수 있음. 다른 기업 눈치보며 서서히 ...,긍정


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 3 to 8824
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      871 non-null    object
 1   감정      871 non-null    object
dtypes: object(2)
memory usage: 20.4+ KB


In [ ]:
df['감정'].value_counts()

긍정    482
부정    389
Name: 감정, dtype: int64

In [ ]:
# 라벨 변경(negative > 0, positive > 1)
def changeTo01(x):
  if x == '부정':
    return 0
  elif x == '긍정':
    return 1

df['감정'] = df['감정'].apply(changeTo01)
df.head()

,문장,감정
3,제공되는 중식으로 식비를 절약할수있음 자유로운 휴식시간과 단순한 문서보조 작업만 해...,1
20,노사다 사관학교 멀티플레어 역할을 할 수있음. 다양한 업무를 경험할 수 있음,1
23,1 많은 토목회사들이 현장근무는 격주 토요일 근무를 하는데 반해 이곳은 주5일 근무...,1
38,좋은 사람들 많습니다 무엇보다 연봉이 높으며 배울 것이 많습니다,1
40,공공발주사업 많이 따와서 업무를 빡세게 배울 수 있음. 다른 기업 눈치보며 서서히 ...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 3 to 8824
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      871 non-null    object
 1   감정      871 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 20.4+ KB


In [ ]:
# [발화문, 상황] data_list 생성
data_list = []

for review, label in zip(df['문장'], df['감정']):
  data = []
  data.append(review)
  data.append(label)
  data_list.append(data)

len(data_list)

871

In [ ]:
# 구축된 데이터셋 확인
print(data)
print(data_list[:10])

['단순보고를 위한 문서가 많아 비효율적이다. 업무 간소화를 한다고 공지만 할뿐 변하지 않는다. 일부부서만 적용한다', 0]
[['제공되는 중식으로 식비를 절약할수있음 자유로운 휴식시간과 단순한 문서보조 작업만 해내면 되는 업무 난이도', 1], ['노사다 사관학교 멀티플레어 역할을 할 수있음. 다양한 업무를 경험할 수 있음', 1], ['1 많은 토목회사들이 현장근무는 격주 토요일 근무를 하는데 반해 이곳은 주5일 근무로 주말은 쉰다2 동종업계 최상위권 연봉은 아니지만 상위권은 되는 연봉3 토목회사가 그렇듯이 자차 소유시 주유대 지원 귀향여비 달마다 지원 1일 3식 모두 지원 및 숙소지원4 대전 1위 건설회사라는 자부심 전국 20위권 이내', 1], ['좋은 사람들 많습니다 무엇보다 연봉이 높으며 배울 것이 많습니다', 1], ['공공발주사업 많이 따와서 업무를 빡세게 배울 수 있음. 다른 기업 눈치보며 서서히 급여 올리는 듯', 1], ['연차 사용이 자유롭고 밥이 맛있습니다 업무 배우기에 좋습니다', 1], ['일 배우기 정말 좋은곳 대리까지 정말 많이 일 배울 수 있는곳', 1], ['그냥 딱 현장경험해보고 경력쌓기가 좋다 공공사업수주가 많아 fm으로 일을 배울 수 있다', 1], ['현장마다 다르지만 개인의 주도하에 공사를 주도할 수 있음본인의 능력 및 의지에 따라 다름', 1], ['많은 것을 배울 수가 있으며 대전의 삼성으로 불리고 있다', 1]]


In [ ]:
# train, test 데이터셋으로 나눔
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32, stratify=df['감정'])
print(len(dataset_train), len(dataset_test))

696 175


## 데이터셋 토큰화

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# PyTorch의 Dataset 클래스를 상속받아 BERT 모델을 학습하기 위한 데이터셋을 생성하는 클래스인 BERTDataset을 정의
# 이 클래스는 데이터셋을 BERT 모델의 입력 형식으로 변환하는 역할

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

BERTDataset 클래스의 파라미터  
* dataset: BERT 모델 학습에 사용될 데이터셋
*  sent_idx: 문장(텍스트)이 있는 열의 인덱스입니다.
* label_idx: 레이블이 있는 열의 인덱스입니다.
* bert_tokenizer: BERT 토크나이저입니다.
* vocab: 어휘 사전입니다.
* max_len: 최대 시퀀스 길이입니다.
* pad: 패딩 여부를 나타내는 불리언 값입니다.
* pair: 문장 페어(pair) 여부를 나타내는 불리언 값입니다.

BERTDataset 클래스의 메서드
* __init__(self, ...): 클래스의 초기화 메서드로, 주어진 매개변수들을 사용하여 데이터셋을 BERT 모델의 입력 형식으로 변환할 수 있는 변환 객체를 생성합니다.
* __getitem__(self, i): 주어진 인덱스 i에 해당하는 데이터셋의 항목을 반환. 반환되는 항목은 BERT 모델의 입력으로 사용될 수 있도록 변환된 문장과 해당 문장의 레이블
* __len__(self): 데이터셋의 전체 길이를 반환

In [ ]:
# 하이퍼파라미터
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tok = tokenizer.tokenize

In [ ]:
train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    else:
        out = pooler
    return self.classifier(out)


In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/11 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.10553777217865 train acc 0.359375
epoch 1 train acc 0.5028409090909091


  0%|          | 0/3 [00:00<?, ?it/s]

epoch 1 test acc 0.5821143617021277


  0%|          | 0/11 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8840868794326241


  0%|          | 0/11 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.21170301735401154 train acc 0.953125
epoch 5 train acc 0.9484577922077921


  0%|          | 0/3 [00:00<?, ?it/s]

epoch 5 test acc 0.9034796099290779


# Prediction

In [ ]:
# 모델 state dict만 저장하기
torch.save(model.state_dict(), "model.pt")
category = {'부정':0,'긍정':1}

In [ ]:
def predict(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]
    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()
          # test_eval.append(list(category.keys())[(np.argmax(logits))]) # 원본 데이터
          test_eval.append(logits)
        # print(">> 입력하신 내용은 '" + test_eval[0] + "'의 카테고리로 예측되었습니다.")
        probabilities = F.softmax(torch.tensor(test_eval[0]), dim=0).numpy()
        # print(">> 입력하신 내용에 대한 확률 값은:", probabilities)


    return probabilities[0], probabilities[1]

In [ ]:
predict('승진 포기하세요')

(0.31005004, 0.67108136)

In [ ]:
review = pd.read_csv('/content/drive/MyDrive/ICR_project/잡플래닛_크롤링/jobplanet_concat.csv')

In [ ]:
# 총평을 담을 리스트
review_list = []

# 총평 문장을 리스트에 하나씩 넣기
for i in review['총평']:
  review_list.append(i)

review_list

In [ ]:
review['score'] = None

In [ ]:
review['score'] = review['총평'].apply(predict)

Exception ignored in: <function _after_fork at 0x7e21572e9990>